In [1]:
import os
import traceback

from hyperbolicTSNE import Datasets, load_data
from hyperbolicTSNE import Datasets
from hyperbolicTSNE.visualization import plot_poincare, animate

Please note that `empty_sequence` uses the KL divergence with Barnes-Hut approximation (angle=0.5) by default.


Testing numpy functionality

In [2]:
import numpy as np
from numpy import linalg as LA 

y = np.array([[1, 1, 1], [2, 2, 2], [3, 3, 3]])
print(y.shape)

yy = (y * y)
print(yy)

norms = yy.sum(axis=1)
print(norms)

(3, 3)
[[1 1 1]
 [4 4 4]
 [9 9 9]]
[ 3 12 27]


Testing global hsne high dim. matrix function

In [3]:
from hyperbolicTSNE.hd_mat_ import globalhsne_D_V

In [2]:
data_home = "datasets"
log_path = "temp/poincare/"  # path for saving embedding snapshots

dataset = Datasets.WORDNET

pca_components = 50 
seed = 42 
perplexity = 50
np = -1

# load data
dataX, dataLabels, D, V, *rest = load_data(
        dataset, 
        data_home=data_home, 
        pca_components=pca_components,
        random_state=seed, 
        to_return="X_labels_D_V",
        hd_params={"perplexity": perplexity}, 
        sample=np, 
        knn_method="hnswlib"  # we use an approximation of high-dimensional neighbors to speed up computations
    )

# Regular p_ij found in D, V

# Determine a good value for nr of neighbours
# TODO: A better way to figure this out? Maybe use quadtree to compute D_hat, V_hat
# n_neighbours = 100

# # get global p_ij (p_ij hat in paper)
# D_hat, V_hat = globalhsne_D_V(dataX, n_neighbours=n_neighbours)

# print(D_hat.shape)
# print(V_hat.shape)

/Users/haoranxia/Thesis-Delft/hyperbolic-tsne/hyperbolicTSNE/data_loaders.py:253: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(str(Path.joinpath(full_pat

In [3]:
print(dataX.shape, dataLabels.shape)

(82115, 11) (82115,)
